# Table of Contents

&emsp;[<a id="#vocab" /> <font color='yellow' size=5> Vocab </font>](#%3Ca-id%3D%22%23vocab%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Vocab-%3C/font%3E)<br>
&emsp;&emsp;[1. Comparison](#1.-Comparison)<br>
&emsp;&emsp;[2. Intensifier](#2.-Intensifier)<br><br>
&emsp;[<a id="Taxonomy" /> <font color='yellow' size=5> Taxonomy </font>](#%3Ca-id%3D%22Taxonomy%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Taxonomy-%3C/font%3E)<br>
&emsp;&emsp;[1. Size, Shape, Color, Age, Material](#1.-Size%2C-Shape%2C-Color%2C-Age%2C-Material)<br>
&emsp;&emsp;[2. Professions vs nationalities](#2.-Professions-vs-nationalities)<br>
&emsp;&emsp;[3. Animal vs Vehicles - Example1](#3.-Animal-vs-Vehicles---Example1)<br>
&emsp;&emsp;[4. Animal vs Vehicles - Example2](#4.-Animal-vs-Vehicles---Example2)<br>
&emsp;&emsp;[5. Synonyms](#5.-Synonyms)<br>
&emsp;&emsp;[6. Antonyms](#6.-Antonyms)<br>
&emsp;&emsp;[7. Antonyms Comparison](#7.-Antonyms-Comparison)<br><br>
&emsp;[<a id="Temporal" /> <font color='yellow' size=5> Temporal </font>](#%3Ca-id%3D%22Temporal%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Temporal-%3C/font%3E)<br>
&emsp;&emsp;[1. Change in profession](#1.-Change-in-profession)<br>
&emsp;&emsp;[2. Understand time difference](#2.-Understand-time-difference)<br><br>
&emsp;[<a id="Negation" /> <font color='yellow' size=5> Negation </font>](#%3Ca-id%3D%22Negation%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Negation-%3C/font%3E)<br>
&emsp;&emsp;[1. Negation in Context](#1.-Negation-in-Context)<br>
&emsp;&emsp;[3. Negation in Question](#3.-Negation-in-Question)<br><br>
&emsp;[<a id="Coref" /> <font color='yellow' size=5> Coref </font>](#%3Ca-id%3D%22Coref%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Coref-%3C/font%3E)<br>
&emsp;&emsp;[1. He/she coref](#1.-He/she-coref)<br>
&emsp;&emsp;[2. His/Her coref](#2.-His/Her-coref)<br>
&emsp;&emsp;[3. Former and Latter](#3.-Former-and-Latter)<br>

In [1]:
%load_ext autoreload
%autoreload 2
import checklist.editor
import munch
import itertools
from checklist.test_suite import TestSuite
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.perturb import Perturb
from helper import get_finetuned_electra_predictor, format_squad_with_context,show_example, crossproduct, export_suite_to_jsonl, get_summary,display_and_export_mdtable, clean, expect_squad,generate_dataset

In [2]:
suite = TestSuite()
editor = checklist.editor.Editor()
model_predictor = get_finetuned_electra_predictor()
expect_squad = Expect.single(expect_squad)

/opt/anaconda3/envs/nlp-proj/lib/python3.10/site-packages/iso639/iso639.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## <a id="#vocab" /> <font color='yellow'> Vocab </font>

### 1. Comparison

In [3]:
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


t_comparison = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    ),
    ],
    labels = ['{first_name1}','{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=800,
    save=True
)

In [4]:
show_example(t_comparison, n=1)

  [1] ('Ruth is slower than Robin.', 'Who is less slow?') , pred: Ruth , label: Robin
  [2] ('Ruth is slower than Robin.', 'Who is slower?') , pred: Ruth , label: Ruth
---


In [5]:
name = 'Comparisons'
description = 'A is COMP than B. Who is more / less COMP?'
test = MFT(**t_comparison, name=name, description=description, capability='Vocabulary')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1592 examples
Test cases:      796
Fails (rate):    796 (100.0%)

Example fails:
C: Lawrence is faster than Barbara.
Q: Who is less fast?
A: Barbara
P: Lawrence


----


### 2. Intensifier

In [6]:
state = editor.suggest('John is very {mask} about the project.')[:20]
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

t_intensifier = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))

/opt/anaconda3/envs/nlp-proj/lib/python3.10/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [7]:
show_example(t_intensifier)

  [1] ('Kathy is highly enthusiastic about the project. Matthew is enthusiastic about the project.', 'Who is most enthusiastic about the project?') , pred: Matthew , label: Kathy
  [2] ('Kathy is highly enthusiastic about the project. Matthew is enthusiastic about the project.', 'Who is least enthusiastic about the project?') , pred: Matthew , label: Matthew
  [3] ('Matthew is enthusiastic about the project. Kathy is highly enthusiastic about the project.', 'Who is most enthusiastic about the project?') , pred: Kathy , label: Kathy
  [4] ('Matthew is enthusiastic about the project. Kathy is highly enthusiastic about the project.', 'Who is least enthusiastic about the project?') , pred: Kathy , label: Matthew
  [5] ('Kathy is enthusiastic about the project. Matthew is somewhat enthusiastic about the project.', 'Who is most enthusiastic about the project?') , pred: Matthew , label: Kathy
  [6] ('Kathy is enthusiastic about the project. Matthew is somewhat enthusiastic about the project.'

In [8]:
name = 'Intensifiers'
desc = '(very, super, extremely) and reducers (somewhat, kinda, etc)?'
test = MFT(**t_intensifier, name=name, description=desc, capability='Vocabulary')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context, n_per_testcase=2)
suite.add(test)

Predicting 9540 examples
Test cases:      795
Fails (rate):    795 (100.0%)

Example fails:
C: Andrea is particularly happy about the project. Robert is happy about the project.
Q: Who is most happy about the project?
A: Andrea
P: Robert

C: Robert is happy about the project. Andrea is particularly happy about the project.
Q: Who is least happy about the project?
A: Robert
P: Andrea


----


## <a id="Taxonomy" /> <font color='yellow'> Taxonomy </font>

### 1. Size, Shape, Color, Age, Material

In [9]:
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']

for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))
            
t_properties = crossproduct(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ), 
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ), 
            
        ]
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))


In [10]:
show_example(t_properties, n=2)

  [1] ('There is a square orange thing in the room.', 'What shape is the thing?') , pred: square orange , label: square
  [2] ('There is a square orange thing in the room.', 'What color is the thing?') , pred: orange , label: orange
  [3] ('There is a thing in the room. The thing is square and orange.', 'What shape is the thing?') , pred: square and orange , label: square
  [4] ('There is a thing in the room. The thing is square and orange.', 'What color is the thing?') , pred: orange , label: orange
---
  [1] ('There is an enormous yellow clock in the room.', 'What size is the clock?') , pred: enormous yellow , label: enormous
  [2] ('There is an enormous yellow clock in the room.', 'What color is the clock?') , pred: yellow , label: yellow
  [3] ('There is a clock in the room. The clock is enormous and yellow.', 'What size is the clock?') , pred: enormous and yellow , label: enormous
  [4] ('There is a clock in the room. The clock is enormous and yellow.', 'What color is the clock?')

In [11]:
name = 'Properites'
desc = 'size, shape, age, color'
test = MFT(**t_properties, name=name, description=desc, capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3200 examples
Test cases:      800
Fails (rate):    797 (99.6%)

Example fails:
C: There is a big round painting in the room.
Q: What size is the painting?
A: big
P: big round

C: There is a painting in the room. The painting is big and round.
Q: What size is the painting?
A: big
P: big and round

C: There is a painting in the room. The painting is big and round.
Q: What shape is the painting?
A: round
P: big and round


----


### 2. Professions vs nationalities

In [12]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

In [13]:
t_profession_nationtionality = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    save=True,
    ))

In [14]:
show_example(t_profession_nationtionality, n=1)

  [1] ('Leslie is a Brazilian historian.', "What is Leslie's job?") , pred: historian , label: historian
  [2] ('Leslie is a Brazilian historian.', "What is Leslie's nationality?") , pred: Brazilian , label: Brazilian
  [3] ('Leslie is a historian. Leslie is Brazilian.', "What is Leslie's job?") , pred: historian , label: historian
  [4] ('Leslie is a historian. Leslie is Brazilian.', "What is Leslie's nationality?") , pred: Brazilian , label: Brazilian
  [5] ('Leslie is Brazilian. Leslie is a historian.', "What is Leslie's job?") , pred: historian , label: historian
  [6] ('Leslie is Brazilian. Leslie is a historian.', "What is Leslie's nationality?") , pred: Brazilian , label: Brazilian
  [7] ('Leslie is Brazilian and a historian.', "What is Leslie's job?") , pred: historian , label: historian
  [8] ('Leslie is Brazilian and a historian.', "What is Leslie's nationality?") , pred: Brazilian , label: Brazilian
  [9] ('Leslie is a historian and Brazilian.', "What is Leslie's job?") , pr

In [15]:
name = 'Profession vs nationality'
test = MFT(**t_profession_nationtionality, name=name, expect=expect_squad, description='The ability to distinguish between professions and nationalities',  capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 8000 examples
Test cases:      800
Fails (rate):    400 (50.0%)

Example fails:
C: Alison is a Japanese analyst.
Q: What is Alison's job?
A: analyst
P: Japanese analyst

C: Alison is an analyst and Japanese.
Q: What is Alison's job?
A: analyst
P: analyst and Japanese


----


### 3. Animal vs Vehicles - Example1

In [16]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t_animal_vehicles = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))

In [17]:
show_example(t_animal_vehicles)

  [1] ('Charlotte has a cat and a truck.', 'What animal does Charlotte have?') , pred: cat , label: cat
  [2] ('Charlotte has a cat and a truck.', 'What vehicle does Charlotte have?') , pred: a cat and a truck , label: truck
  [3] ('Charlotte has a truck and a cat.', 'What animal does Charlotte have?') , pred: cat , label: cat
  [4] ('Charlotte has a truck and a cat.', 'What vehicle does Charlotte have?') , pred: a truck and a cat , label: truck
---


In [18]:
name = 'Animal vs Vehicle'
test = MFT(**t_animal_vehicles, name=name, description='The ability to distinguish between animals and vehicles by asking about the object (animal or vehicle)', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 3200 examples
Test cases:      800
Fails (rate):    796 (99.5%)

Example fails:
C: Kate has a lizard and a minivan.
Q: What vehicle does Kate have?
A: minivan
P: a lizard and a minivan


----


### 4. Animal vs Vehicles - Example2

In [19]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t_animals_vehicles_2 = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} bought {a:animal}. {first_name2} bought {a:vehicle}.',
            '{first_name2} bought {a:vehicle}. {first_name} bought {a:animal}.',
        ],
        'qas': [
            (
                'Who bought an animal?',
                '{first_name}'
            ), 
            (
                'Who bought a vehicle?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))


In [20]:
show_example(t_animals_vehicles_2)

  [1] ('Ruth bought a fish. Steven bought a van.', 'Who bought an animal?') , pred: Steven , label: Ruth
  [2] ('Ruth bought a fish. Steven bought a van.', 'Who bought a vehicle?') , pred: Steven , label: Steven
  [3] ('Steven bought a van. Ruth bought a fish.', 'Who bought an animal?') , pred: Ruth , label: Ruth
  [4] ('Steven bought a van. Ruth bought a fish.', 'Who bought a vehicle?') , pred: Steven , label: Steven
---


In [21]:

name = 'Animal vs Vehicle v2'
test = MFT(**t_animals_vehicles_2, name=name, description='The ability to distinguish between animals and vehicles by asking about who bougth the object', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 3172 examples
Test cases:      793
Fails (rate):    413 (52.1%)

Example fails:
C: Joan bought a serpent. Charles bought a van.
Q: Who bought an animal?
A: Joan
P: Charles


----


### 5. Synonyms 

In [22]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

t_synonym = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[0]}. {first_name2} is very {s2[0]}.',
            '{first_name2} is very {s2[0]}. {first_name} is very {s1[0]}.',
        ],
        'qas': [
            (
                'Who is {s1[1]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[1]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=400,
    save=True
   ))
t_synonym += crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[1]}. {first_name2} is very {s2[1]}.',
            '{first_name2} is very {s2[1]}. {first_name} is very {s1[1]}.',
        ],
        'qas': [
            (
                'Who is {s1[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[0]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=400,
    save=True
    )) 

In [23]:
show_example(t_synonym)

  [1] ('Richard is very grateful. Nick is very angry.', 'Who is thankful?') , pred: Richard , label: Richard
  [2] ('Richard is very grateful. Nick is very angry.', 'Who is furious?') , pred: Nick , label: Nick
  [3] ('Nick is very angry. Richard is very grateful.', 'Who is thankful?') , pred: Richard , label: Richard
  [4] ('Nick is very angry. Richard is very grateful.', 'Who is furious?') , pred: Nick , label: Nick
---


In [24]:
name = 'Synonyms'
test = MFT(**t_synonym, name=name, description='The ability to distinguish between synonyms', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 2888 examples
Test cases:      722
Fails (rate):    47 (6.5%)

Example fails:
C: Arthur is very vocal. Kenneth is very courageous.
Q: Who is outspoken?
A: Arthur
P: Kenneth


----


### 6. Antonyms

In [25]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))

In [26]:
t_antonymns = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))

In [27]:
show_example(t_antonymns)

  [1] ('Sarah is faster than Kelly.', 'Who is slower?') , pred: Sarah , label: Kelly
  [2] ('Sarah is faster than Kelly.', 'Who is faster?') , pred: Sarah , label: Sarah
  [3] ('Kelly is slower than Sarah.', 'Who is slower?') , pred: Kelly , label: Kelly
  [4] ('Kelly is slower than Sarah.', 'Who is faster?') , pred: Kelly , label: Sarah
---


In [28]:
name = 'Antonyms'
test = MFT(**t_antonymns, name=name, description='A is COMP than B. Who is antonym(COMP)? B', capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3168 examples
Test cases:      792
Fails (rate):    792 (100.0%)

Example fails:
C: Heather is warmer than Emma.
Q: Who is colder?
A: Emma
P: Heather

C: Emma is colder than Heather.
Q: Who is warmer?
A: Heather
P: Emma


----


### 7. Antonyms Comparison

In [29]:
antonym_adjs = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]

t_antonymns_compare = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is more {a[0]} than {first_name1}.',
            '{first_name1} is more {a[1]} than {first_name}.',
            '{first_name} is less {a[1]} than {first_name1}.',
            '{first_name1} is less {a[0]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is more {a[0]}?',
                '{first_name}',
            ),
            (
                'Who is less {a[0]}?',
                '{first_name1}',
            ),
            (
                'Who is more {a[1]}?',
                '{first_name1}',
            ),
            (
                'Who is less {a[1]}?',
                '{first_name}',
            ),
        ]
        ,
    },
    a = antonym_adjs,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))

In [30]:
show_example(t_antonymns_compare)

  [1] ('Ray is more pessimistic than Ralph.', 'Who is more pessimistic?') , pred: Ray , label: Ray
  [2] ('Ray is more pessimistic than Ralph.', 'Who is less pessimistic?') , pred: Ray , label: Ralph
  [3] ('Ray is more pessimistic than Ralph.', 'Who is more optimistic?') , pred: Ray , label: Ralph
  [4] ('Ray is more pessimistic than Ralph.', 'Who is less optimistic?') , pred: Ray , label: Ray
  [5] ('Ralph is more optimistic than Ray.', 'Who is more pessimistic?') , pred: Ralph , label: Ray
  [6] ('Ralph is more optimistic than Ray.', 'Who is less pessimistic?') , pred: Ralph , label: Ralph
  [7] ('Ralph is more optimistic than Ray.', 'Who is more optimistic?') , pred: Ralph , label: Ralph
  [8] ('Ralph is more optimistic than Ray.', 'Who is less optimistic?') , pred: Ralph , label: Ray
  [9] ('Ray is less optimistic than Ralph.', 'Who is more pessimistic?') , pred: Ray , label: Ray
  [10] ('Ray is less optimistic than Ralph.', 'Who is less pessimistic?') , pred: Ray , label: Ralph
 

In [31]:
description = 'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.'
name= 'Antonyms Comparison'
test = MFT(**t_antonymns_compare, name=name, description=description, capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 12752 examples
Test cases:      797
Fails (rate):    797 (100.0%)

Example fails:
C: Katie is more rude than Lawrence.
Q: Who is less rude?
A: Lawrence
P: Katie

C: Katie is more rude than Lawrence.
Q: Who is more polite?
A: Lawrence
P: Katie

C: Lawrence is more polite than Katie.
Q: Who is more rude?
A: Katie
P: Lawrence


----


## <a id="Temporal" /> <font color='yellow'> Temporal </font>

### 1. Change in profession

In [32]:
t_profession = crossproduct(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    ))


In [33]:
show_example(t_profession)

  [1] ('Both Patrick and Jane were producers, but there was a change in Patrick, who is now an assistant.', 'Who is an assistant?') , pred: Patrick , label: Patrick
  [2] ('Both Jane and Patrick were producers, but there was a change in Patrick, who is now an assistant.', 'Who is an assistant?') , pred: Patrick , label: Patrick
---


In [34]:
name = 'Change in profession'
test = MFT(**t_profession, expect=expect_squad, capability='Temporal', name=name, description='The ability to detect a change in profession over time' )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1540 examples
Test cases:      770
Fails (rate):    7 (0.9%)

Example fails:
C: Both Stephen and Carol were architects, but there was a change in Carol, who is now an analyst.
Q: Who is an analyst?
A: Carol
P: empty


----


### 2. Understand time difference
e.g before, after

In [35]:
t_time_difference = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ), 
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    ))

In [36]:
show_example(t_time_difference)

  [1] ('Sara became a actress before Colin did.', 'Who became a actress first?') , pred: Sara , label: Sara
  [2] ('Sara became a actress before Colin did.', 'Who became a actress last?') , pred: Sara , label: Colin
  [3] ('Colin became a actress after Sara did.', 'Who became a actress first?') , pred: Colin , label: Sara
  [4] ('Colin became a actress after Sara did.', 'Who became a actress last?') , pred: Colin , label: Colin
---


In [37]:
description = 'Understanding before / after -> first / last.'
name="Time Difference"
test = MFT(**t_time_difference, expect=expect_squad, capability='Temporal', name=name, description=description )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3188 examples
Test cases:      797
Fails (rate):    797 (100.0%)

Example fails:
C: Wendy became a educator before Kevin did.
Q: Who became a educator last?
A: Kevin
P: Wendy

C: Kevin became a educator after Wendy did.
Q: Who became a educator first?
A: Wendy
P: Kevin


----


## <a id="Negation" /> <font color='yellow'> Negation </font>

### 1. Negation in Context

In [38]:
t_context_negation = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    ))

In [39]:
show_example(t_context_negation)

  [1] ('Charles is not a producer. Ben is.', 'Who is a producer?') , pred: Charles , label: Ben
  [2] ('Charles is not a producer. Ben is.', 'Who is not a producer?') , pred: Charles , label: Charles
  [3] ('Ben is a producer. Charles is not.', 'Who is a producer?') , pred: Ben , label: Ben
  [4] ('Ben is a producer. Charles is not.', 'Who is not a producer?') , pred: Charles , label: Charles
---


In [40]:

name = 'Negation in context, may or may not be in question'
test = MFT(**t_context_negation, expect=expect_squad, capability='Negation', name=name, description='Ability to handle negation in context, whether or not it appears in the question' )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3188 examples
Test cases:      797
Fails (rate):    784 (98.4%)

Example fails:
C: Lawrence is not an artist. Joe is.
Q: Who is an artist?
A: Joe
P: empty


----


### 3. Negation in Question

In [41]:
t_question_negation = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    ))


In [42]:

show_example(t_question_negation)

  [1] ('George is an architect. Lynn is a historian.', 'Who is an architect?') , pred: George , label: George
  [2] ('George is an architect. Lynn is a historian.', 'Who is not an architect?') , pred: George , label: Lynn
  [3] ('George is an architect. Lynn is a historian.', 'Who is a historian?') , pred: Lynn , label: Lynn
  [4] ('George is an architect. Lynn is a historian.', 'Who is not a historian?') , pred: Lynn , label: George
  [5] ('Lynn is a historian. George is an architect.', 'Who is an architect?') , pred: George , label: George
  [6] ('Lynn is a historian. George is an architect.', 'Who is not an architect?') , pred: George , label: Lynn
  [7] ('Lynn is a historian. George is an architect.', 'Who is a historian?') , pred: Lynn , label: Lynn
  [8] ('Lynn is a historian. George is an architect.', 'Who is not a historian?') , pred: Lynn , label: George
---


In [43]:
name = 'Negation in question only.'
test = MFT(**t_question_negation, expect=expect_squad, capability='Negation', name=name, description='The ability to handle negation when it appears only in the question' )
test.run(model_predictor)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 6192 examples
Test cases:      774
Fails (rate):    774 (100.0%)

Example fails:
C: Bruce is an editor. Al is a nurse.
Q: Who is not an editor?
A: Al
P: Bruce

C: Bruce is an editor. Al is a nurse.
Q: Who is not a nurse?
A: Bruce
P: Al

C: Al is a nurse. Bruce is an editor.
Q: Who is not an editor?
A: Al
P: Bruce


----
C: Rebecca is a journalist. Deborah is a musician.
Q: Who is not a journalist?
A: Deborah
P: Rebecca

C: Rebecca is a journalist. Deborah is a musician.
Q: Who is not a musician?
A: Rebecca
P: Deborah

C: Deborah is a musician. Rebecca is a journalist.
Q: Who is not a journalist?
A: Deborah
P: Rebecca


----
C: Sue is an actor. Patricia is an entrepreneur.
Q: Who is not an actor?
A: Patricia
P: Sue

C: Sue is an actor. Patricia is an entrepreneur.
Q: Who is not an entrepreneur?
A: Sue
P: Patricia

C: Patricia is an entrepreneur. Sue is an actor.
Q: Who is not an actor?
A: Patricia
P: Sue


----


## <a id="Coref" /> <font color='yellow'> Coref </font>

### 1. He/she coref

In [44]:
if 'actress' in professions:
    professions.remove('actress')

t_he_she_coref = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ), 
            (
                'Who is {a:prof2}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    ))


In [45]:
show_example(t_he_she_coref)

  [1] ('Paul and Claire are friends. He is a writer, and she is a secretary.', 'Who is a writer?') , pred: Paul and Claire , label: Paul
  [2] ('Paul and Claire are friends. He is a writer, and she is a secretary.', 'Who is a secretary?') , pred: Paul and Claire , label: Claire
  [3] ('Claire and Paul are friends. He is a writer, and she is a secretary.', 'Who is a writer?') , pred: Claire and Paul , label: Paul
  [4] ('Claire and Paul are friends. He is a writer, and she is a secretary.', 'Who is a secretary?') , pred: Claire and Paul , label: Claire
  [5] ('Paul and Claire are friends. She is a secretary, and he is a writer.', 'Who is a writer?') , pred: Paul and Claire , label: Paul
  [6] ('Paul and Claire are friends. She is a secretary, and he is a writer.', 'Who is a secretary?') , pred: Paul and Claire , label: Claire
  [7] ('Claire and Paul are friends. She is a secretary, and he is a writer.', 'Who is a writer?') , pred: Claire and Paul , label: Paul
  [8] ('Claire and Paul ar

In [46]:
name = 'Basic coref, he / she'
test = MFT(**t_he_she_coref, expect=expect_squad, name=name, description='Ability to handle basic coreference with he and she given names in context', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 6224 examples
Test cases:      778
Fails (rate):    778 (100.0%)

Example fails:
C: David and Kelly are friends. He is an entrepreneur, and she is a secretary.
Q: Who is an entrepreneur?
A: David
P: David and Kelly

C: David and Kelly are friends. He is an entrepreneur, and she is a secretary.
Q: Who is a secretary?
A: Kelly
P: David and Kelly

C: Kelly and David are friends. He is an entrepreneur, and she is a secretary.
Q: Who is an entrepreneur?
A: David
P: Kelly and David


----


### 2. His/Her coref

In [47]:
t_his_her = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. His mom is {a:prof}.',
            '{female} and {male} are friends. His mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{male}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=400,
    ))
t_his_her += crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. Her mom is {a:prof}.',
            '{female} and {male} are friends. Her mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=400,
    ))


In [48]:
show_example(t_his_her)

  [1] ('Matthew and Leslie are friends. His mom is an investigator.', 'Whose mom is an investigator?') , pred: Matthew , label: Matthew
  [2] ('Leslie and Matthew are friends. His mom is an investigator.', 'Whose mom is an investigator?') , pred: Leslie and Matthew , label: Matthew
---


In [49]:
name = 'Basic coref, his / her'
test = MFT(**t_his_her, expect=expect_squad, name=name, description='Ability to handle basic coreference with his and her given names in context', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1600 examples
Test cases:      800
Fails (rate):    800 (100.0%)

Example fails:
C: Howard and Lauren are friends. Her mom is an actor.
Q: Whose mom is an actor?
A: Lauren
P: Howard

C: Lauren and Howard are friends. Her mom is an actor.
Q: Whose mom is an actor?
A: Lauren
P: Lauren and Howard


----


### 3. Former and Latter

In [50]:
t_former_latter = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=800,
    save=True
    ))


In [51]:
show_example(t_former_latter)

  [1] ('Bruce and Judith are friends. The former is an activist.', 'Who is an activist?') , pred: Bruce and Judith , label: Bruce
  [2] ('Judith and Bruce are friends. The latter is an activist.', 'Who is an activist?') , pred: Judith and Bruce , label: Bruce
  [3] ('Bruce and Judith are friends. The former is an activist and the latter is a DJ.', 'Who is an activist?') , pred: Bruce and Judith , label: Bruce
  [4] ('Judith and Bruce are friends. The former is a DJ and the latter is an activist.', 'Who is an activist?') , pred: Judith and Bruce , label: Bruce
---


In [52]:
name = 'Former / Latter'
test = MFT(**t_former_latter, expect=expect_squad, name=name, description='Ability to handle coreference with former and latter given names in context', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3068 examples
Test cases:      767
Fails (rate):    767 (100.0%)

Example fails:
C: Keith and Kenneth are friends. The former is an analyst.
Q: Who is an analyst?
A: Keith
P: Keith and Kenneth

C: Kenneth and Keith are friends. The latter is an analyst.
Q: Who is an analyst?
A: Keith
P: Kenneth and Keith

C: Keith and Kenneth are friends. The former is an analyst and the latter is an educator.
Q: Who is an analyst?
A: Keith
P: Keith and Kenneth


----


In [53]:
df_suite=get_summary(suite)

In [54]:
display_and_export_mdtable(df_suite, do_export=False)

| Test Name                                          |   Total Cases |   Example Per Case |   Failures | Failure Rate   |
|----------------------------------------------------|---------------|--------------------|------------|----------------|
| Comparisons                                        |           796 |                  2 |        796 | 100.00%        |
| Intensifiers                                       |           795 |                 12 |        795 | 100.00%        |
| Properites                                         |           800 |                  4 |        797 | 99.62%         |
| Profession vs nationality                          |           800 |                 10 |        400 | 50.00%         |
| Animal vs Vehicle                                  |           800 |                  4 |        796 | 99.50%         |
| Animal vs Vehicle v2                               |           793 |                  4 |        413 | 52.08%         |
| Synonyms                                           |           722 |                  4 |         47 | 6.51%          |
| Antonyms                                           |           792 |                  4 |        792 | 100.00%        |
| Antonyms Comparison                                |           797 |                 16 |        797 | 100.00%        |
| Change in profession                               |           770 |                  2 |          7 | 0.91%          |
| Time Difference                                    |           797 |                  4 |        797 | 100.00%        |
| Negation in context, may or may not be in question |           797 |                  4 |        784 | 98.37%         |
| Negation in question only.                         |           774 |                  8 |        774 | 100.00%        |
| Basic coref, he / she                              |           778 |                  8 |        778 | 100.00%        |
| Basic coref, his / her                             |           800 |                  2 |        800 | 100.00%        |
| Former / Latter                                    |           767 |                  4 |        767 | 100.00%        |

In [55]:
generate_dataset(suite=suite,output_folder="ChecklistData", train_ratio=0.7)

✅ Saved, 72512 rows to ChecklistData/temp_combined.jsonl.
🔒 Generated 72512 unique IDs (collision-free)
📊 Splitting each test separately with 70.0% train ratio:
   Comparisons: 557/796 groups train (70.0% examples)
   Intensifiers: 556/795 groups train (69.9% examples)
   Properites: 560/800 groups train (70.0% examples)
   Profession vs nationality: 560/800 groups train (70.0% examples)
   Animal vs Vehicle: 560/800 groups train (70.0% examples)
   Animal vs Vehicle v2: 555/793 groups train (70.0% examples)
   Synonyms: 505/722 groups train (69.9% examples)
   Antonyms: 554/792 groups train (69.9% examples)
   Antonyms Comparison: 558/797 groups train (70.0% examples)
   Change in profession: 539/770 groups train (70.0% examples)
   Time Difference: 558/797 groups train (70.0% examples)
   Negation in context, may or may not be in question: 558/797 groups train (70.0% examples)
   Negation in question only.: 542/774 groups train (70.0% examples)
   Basic coref, he / she: 545/778 group

([{'id': 'b3ae7c322ccc49e5bb831c84',
   'title': 'Comparisons',
   'context': 'Howard is greater than Charlie.',
   'question': 'Who is less great?',
   'answers': {'text': ['Charlie'], 'answer_start': [23]},
   'pred': 'Howard',
   'test_details': {'test_type': 'MFT',
    'capability': 'Vocabulary',
    'description': 'A is COMP than B. Who is more / less COMP?',
    'name': 'Comparisons',
    'testcase_id': 568,
    'qa_pair_index': 0,
    'total_qa_pairs': 2,
    'expect_function': 'expect'}},
  {'id': '3948c9facafe44a5a6fa6975',
   'title': 'Comparisons',
   'context': 'Howard is greater than Charlie.',
   'question': 'Who is greater?',
   'answers': {'text': ['Howard'], 'answer_start': [0]},
   'pred': 'Howard',
   'test_details': {'test_type': 'MFT',
    'capability': 'Vocabulary',
    'description': 'A is COMP than B. Who is more / less COMP?',
    'name': 'Comparisons',
    'testcase_id': 568,
    'qa_pair_index': 1,
    'total_qa_pairs': 2,
    'expect_function': 'expect'}},
 